# Census Income - Proyect with Random Forest (Modelo 5)

> Aqui vamos a equiparar a la clase minoritaria submuestrando la clase mayoritaria para que entrenar el modelo con una cantidad de clases relativamente equiparables

> Recordar que al disminuir datos de las clase mayoritaria se esta perdiendo informacion de esta

Variables del conjunto de datos "Census Income" de UCI. Este conjunto de datos se utiliza comúnmente para tareas de clasificación, como predecir si una persona gana más o menos de $50,000 al año, en base a sus características. Las caracteristicas son:

1. **age** (edad): La edad de la persona en años.
2. **workclass** (tipo de trabajo): Clasificación del tipo de empleo (e.g., asalariado, autónomo, gobierno).
3. **fnlwgt** (ponderación final): Peso final calculado para las muestras en la encuesta, representa el número de personas que una fila puede representar.
4. **education** (educación): Nivel educativo alcanzado (e.g., escuela secundaria, licenciatura).
5. **education-num** (número de años de educación): Número de años de educación recibidos.
6. **marital-status** (estado civil): Estado civil de la persona (e.g., casado, soltero).
7. **occupation** (ocupación): Tipo de ocupación o profesión de la persona.
8. **relationship** (relación): Relación de la persona con el jefe de familia (e.g., esposo, esposa, hijo).
9. **race** (raza): Clasificación racial de la persona (e.g., blanca, negra, asiática).
10. **sex** (sexo): Sexo de la persona (masculino o femenino).
11. **capital-gain** (ganancia de capital): Ganancias de capital obtenidas por inversiones.
12. **capital-loss** (pérdida de capital): Pérdidas de capital incurridas.
13. **hours-per-week** (horas por semana): Número de horas trabajadas por semana.
14. **native-country** (país de origen): País de nacimiento de la persona.
15. **income** (ingreso): Ingreso anual, clasificado en ">50K" o "<=50K", indicando si la persona gana más o menos de $50,000 al año.



In [27]:
import pandas as pd

# Cargar el archivo de datos principal 'adult.data'
data_path = './adult.data'
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 
                'marital_status', 'occupation', 'relationship', 'race', 
                'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 
                'native_country', 'income']

# Cargar el dataset en un DataFrame
df = pd.read_csv(data_path, header=None, names=column_names, engine='python')

# Mostrar las primeras filas
df.head(2)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [29]:
# Equiparando datos
df_new = df[df.income == ' <=50K']
df_normal = df[df.income == ' >50K']

# Extraer 'n' filas aleatorias
n_filas = 8000  # Elige la cantidad de filas que deseas extraer

# Extraer 'n' filas aleatorias con una semilla
df_muestra = df_new.sample(n= n_filas, random_state=69)

df_model = pd.concat([df_muestra, df_normal], ignore_index=True)

# Mezclar los datos del DataFrame
df_model = df_model.sample(frac=1, random_state=42).reset_index(drop=True)


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

X = df_model.drop(['income','native_country','race'], axis=1)
y = df_model['income']

ord_enc = OrdinalEncoder()
X = pd.DataFrame(ord_enc.fit_transform(X), columns=X.columns)

In [31]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

rf = RandomForestClassifier(bootstrap=True,
                            max_depth=20,
                            max_features= 5,
                            min_samples_leaf = 6,
                            min_samples_split= 2,
                            n_estimators= 200,
                            # class_weight = {' <=50K': 0.7, ' >50K': 1},

    random_state=42)
rf
# entrenamos el modelo
rf.fit(x_train, y_train)
rf

RandomForestClassifier(max_depth=20, max_features=5, min_samples_leaf=6,
                       n_estimators=200, random_state=42)

In [32]:
from sklearn.metrics import classification_report
y_pred = rf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       <=50K       0.85      0.78      0.81      2434
        >50K       0.79      0.85      0.82      2319

    accuracy                           0.82      4753
   macro avg       0.82      0.82      0.82      4753
weighted avg       0.82      0.82      0.82      4753



> En primera instancia podemos apreciar una mejora en la precision y recall para la clase anteriormente minoritaria pero que en este caso esta balanceada con la otra clase

In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1909,  525],
       [ 350, 1969]], dtype=int64)

In [50]:
# Cargar el dataset original en un DataFrame
df = pd.read_csv(data_path, header=None, names=column_names, engine='python')

X_origin = df.drop(['income','native_country','race'], axis=1)
y_origin = df['income']

ord_enc_origin = OrdinalEncoder()
X_origin = pd.DataFrame(ord_enc.fit_transform(X_origin), columns=X.columns)


In [51]:

y_pred_origin = rf.predict(X_origin)

print(classification_report(y_origin, y_pred_origin))


              precision    recall  f1-score   support

       <=50K       0.96      0.79      0.87     24720
        >50K       0.57      0.89      0.70      7841

    accuracy                           0.81     32561
   macro avg       0.76      0.84      0.78     32561
weighted avg       0.86      0.81      0.82     32561



In [48]:
confusion_matrix(y_origin, y_pred_origin)

array([[19531,  5189],
       [  895,  6946]], dtype=int64)

## Conclusiones

- Testeando este nuevo modelo con los datos originales podemos visualizar un aumento en la cantidad de predicciones para la clase minoritaria, sin embargo la perdida de informacion producto del submuestreo nos llevo a una prediccion mas sesgada para la clase mayoritaria, ademas de una cantidad alta de falsos positivos, lo cual de nuevo no compensa el costo

- A pesar de los intentos por mejorar la prediccion para la clase minoritaria, el modelo 3 esta presentando un mejor performance para ambas clases, sin perder informacion